In [172]:
import ast  # for converting embeddings saved as strings back to arrays
import openai # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search

In [173]:
# Configuración de OpenAI
openai.api_key = "sk-proj-JEQe19qf1majBtkGMsubT3BlbkFJeGguQ7FTMjap1uVkFViu"
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 4096

In [174]:
def num_tokens(text: str, model: str = EMBEDDING_MODEL) -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [175]:
def create_embeddings_from_txt_files(directory: str, model: str = EMBEDDING_MODEL, max_tokens: int = MAX_TOKENS) -> pd.DataFrame:
    texts = []
    embeddings = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                text = file.read()
                
                # Split the text into manageable chunks
                token_count = num_tokens(text, model=model)
                if token_count > max_tokens:
                    print(f"Warning: File {filename} exceeds max token limit and will be truncated.")
                    text = text[:max_tokens]
                
                response = openai.Embedding.create(model=model, input=text)
                embedding = response['data'][0]['embedding']
                
                texts.append(text)
                embeddings.append(embedding)
    
    df = pd.DataFrame({"text": texts, "embedding": embeddings})
    return df

In [176]:
# Uso de la función
directory = "data"
df = create_embeddings_from_txt_files(directory)
print(df.head())

                                                text  \
0  Cálculo del consumo de combustible\n\nAl deter...   
1  Los resultados se muestran en dos paneles: la ...   
2   Primero, se selecciona la plantilla de inform...   
3  En la pestaña de informes, se pueden crear inf...   

                                           embedding  
0  [-0.008142594248056412, 0.00439767399802804, 0...  
1  [-0.016902418807148933, 0.006421457510441542, ...  
2  [-0.02123759128153324, -0.013756769709289074, ...  
3  [-0.0005416622734628618, 0.005904242396354675,...  


In [177]:
# Función para clasificar las strings por relevancia usando embeddings
def strings_ranked_by_relatedness(query: str, df: pd.DataFrame, top_n: int = 5, threshold: float = 0.76) -> list[tuple[str, float]]:
    query_embedding_response = openai.Embedding.create(model=EMBEDDING_MODEL, input=query)
    query_embedding = query_embedding_response['data'][0]['embedding']
    strings_and_relatednesses = [(row["text"], 1 - spatial.distance.cosine(query_embedding, row["embedding"])) for _, row in df.iterrows()]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    valid_sections = [(s, r) for s, r in strings_and_relatednesses if r >= threshold]
    
    return valid_sections[:top_n]

In [178]:
# Función para crear el mensaje de consulta
def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int, threshold: float = 0.76) -> str:
    valid_sections = strings_ranked_by_relatedness(query, df, threshold=threshold)
    
    if len(valid_sections) == 0:
        return "No cuento con esa información, por favor contactar a servicio a cliente."
    
    introduction = 'Use the below articles to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string, relatedness in valid_sections:
        next_article = f'\n\nText section:\n"""\n{string}\n"""'
        if num_tokens(message + next_article + question, model=model) > token_budget:
            break
        else:
            message += next_article
    return message + question

In [179]:
# Función para hacer la consulta a GPT
def ask(query: str, df: pd.DataFrame, model: str = GPT_MODEL, token_budget: int = 4096 - 500, print_message: bool = False, threshold: float = 0.76) -> str:
    message = query_message(query, df, model=model, token_budget=token_budget, threshold=threshold)
    if message == "No cuento con esa información, por favor contactar a servicio a cliente.":
        return message
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions based on the provided articles."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)
    return response['choices'][0]['message']['content']

In [180]:
def print_formatted_response(response: str) -> None:
    formatted_response = response.replace("\\n", "\n")
    formatted_response = formatted_response.replace('Wialon', 'Quamtum')
    return formatted_response

In [181]:
# Ejemplo de uso
query = "Hablame de los informes"
response = ask(query, df, model=GPT_MODEL, print_message=False, threshold=0.76)
print(print_formatted_response(response))

Los informes son herramientas que permiten analizar datos de seguimiento de una manera personalizada. Se pueden crear informes personalizados para analizar información específica, como en el caso de un informe de estacionamiento. Estos informes constan de tres secciones: contenido, ajustes y asignación. En el contenido del informe se incluye información relevante como el nombre del informe, grupos de unidades, intervalo de tiempo y tiempo de ejecución. En los ajustes del informe se definen aspectos como la agrupación de datos, las columnas a mostrar y la configuración del mapa. Una vez configurado el informe, se puede ejecutar para generar los resultados deseados, los cuales se mostrarán en el panel de resultados de informes. Los informes pueden exportarse a diferentes formatos, como PDF, Excel, HTML, XML o CSV, e incluso imprimirse.
